In [ ]:
from flask import Flask, render_template, request, redirect, url_for, flash, session
import mysql.connector

app = Flask(__name__)
app.secret_key = "Shree"  # Ensure this is unique for session management

def get_connection():
    return mysql.connector.connect(
        host='mydbinstance.cpkukqi8wb7l.eu-north-1.rds.amazonaws.com',
        port=3306,
        user='Bhavesh',
        password='bhavesh1975',
        database='Dream_House_Valuation',
        connection_timeout=10
    )

# Create table if not exists
def setup_database():
    connection = get_connection()
    cursor = connection.cursor()
    cursor.execute("CREATE TABLE IF NOT EXISTS registers (Email VARCHAR(70), Full_Name CHAR(70), mobile VARCHAR(15), password VARCHAR(20))")
    connection.commit()
    cursor.close()
    connection.close()

setup_database()

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/interaction')
def interaction():
    if 'user_email' not in session:
        flash("Please log in first.")
        return redirect(url_for('home'))

    email = session['user_email']

    connection = get_connection()
    cursor = connection.cursor()
    cursor.execute("SELECT Full_Name, Email FROM registers WHERE Email = %s", (email,))
    user = cursor.fetchone()
    cursor.close()
    connection.close()

    if user:
        user_info = {
            'name': user[0],
            'email': user[1]
        }
        return render_template('interaction2.html', user=user_info)
    else:
        flash("User not found.")
        return redirect(url_for('home'))

@app.route('/register', methods=['POST'])
def register():
    full_name = request.form['fullname']
    mobile = request.form['mobile']
    email = request.form['email']
    password = request.form['password']
    confirm_password = request.form['confirm_password']

    if password != confirm_password:
        flash("Passwords do not match.")
        return redirect(url_for('home'))

    try:
        connection = get_connection()
        cursor = connection.cursor()
        cursor.execute(
            "INSERT INTO registers (Full_Name, mobile, Email, password) VALUES (%s, %s, %s, %s)",
            (full_name, mobile, email, password)
        )
        connection.commit()
        flash("Registration successful.")
    except Exception as e:
        flash(f"Error: {e}")
    finally:
        if cursor:
            cursor.close()
        if connection:
            connection.close()

    return redirect(url_for('home'))


@app.route('/login', methods=['POST'])
def login():
    email = request.form['email']
    password = request.form['password']

    connection = None
    cursor = None
    result = None

    try:
        connection = get_connection()
        cursor = connection.cursor()
        cursor.execute("SELECT * FROM registers WHERE Email = %s AND password = %s", (email, password))
        result = cursor.fetchone()

        print(f"Query result: {result}")  # Debugging line to check if the query is returning expected data
    except Exception as e:
        flash(f"Error: {e}")
        return redirect(url_for('home'))
    finally:
        if cursor:
            try:
                cursor.close()
            except Exception as e:
                print(f"Cursor close error: {e}")
        if connection:
            try:
                connection.close()
            except Exception as e:
                print(f"Connection close error: {e}")

    if result:
        session['user_email'] = result[0]  # Assuming Email is at index 0 in result
        flash(f"Welcome {result[1]}")  # Assuming Full_Name is at index 1
        return redirect(url_for('interaction'))
    else:
        flash("Invalid email or password.")
        return redirect(url_for('home'))

@app.route('/logout')
def logout():
    session.pop('user_email', None)
    flash("Logged out successfully.")
    return redirect(url_for('home'))


if __name__ == '__main__':
    app.run(debug=True, use_reloader=False)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [18/Apr/2025 11:20:02] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/Apr/2025 11:20:02] "GET /static/css/adjustment2.css HTTP/1.1" 304 -
127.0.0.1 - - [18/Apr/2025 11:20:02] "GET /static/images/logo.png HTTP/1.1" 304 -
127.0.0.1 - - [18/Apr/2025 11:20:02] "GET /static/images/Designer%201.jpeg HTTP/1.1" 304 -
127.0.0.1 - - [18/Apr/2025 11:20:03] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [18/Apr/2025 11:20:25] "POST /login HTTP/1.1" 302 -
127.0.0.1 - - [18/Apr/2025 11:20:25] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/Apr/2025 11:20:25] "GET /static/css/adjustment2.css HTTP/1.1" 304 -
127.0.0.1 - - [18/Apr/2025 11:20:25] "GET /static/images/logo.png HTTP/1.1" 304 -
127.0.0.1 - - [18/Apr/2025 11:20:50] "POST /login HTTP/1.1" 302 -


Query result: ('bsbhavesh1975@gmail.com', 'bhavesh shah', '@bhavesh1975', None)


127.0.0.1 - - [18/Apr/2025 11:21:13] "GET /interaction HTTP/1.1" 200 -
127.0.0.1 - - [18/Apr/2025 11:21:14] "GET /static/css/adjustment2.css HTTP/1.1" 304 -
127.0.0.1 - - [18/Apr/2025 11:21:14] "GET /static/images/logo.png HTTP/1.1" 304 -
127.0.0.1 - - [18/Apr/2025 11:21:14] "GET /static/images/profile.jpeg HTTP/1.1" 304 -
